# Quick EDA - Santa 2025 Tree Packing

In [1]:
import pandas as pd
import numpy as np

# Load sample submission
df = pd.read_csv('/home/data/sample_submission.csv')
print(f'Shape: {df.shape}')
print(df.head(10))

Shape: (20100, 4)
      id           x           y     deg
0  001_0        s0.0        s0.0   s90.0
1  002_0        s0.0        s0.0   s90.0
2  002_1   s0.202736  s-0.511271   s90.0
3  003_0        s0.0        s0.0   s90.0
4  003_1   s0.202736  s-0.511271   s90.0
5  003_2     s0.5206   s0.177413  s180.0
6  004_0        s0.0        s0.0   s90.0
7  004_1   s0.202736  s-0.511271   s90.0
8  004_2     s0.5206   s0.177413  s180.0
9  004_3  s-0.818657  s-0.228694  s180.0


In [ ]:
# Calculate score for sample submission
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal('1')

# Tree geometry
trunk_w = Decimal('0.15')
trunk_h = Decimal('0.2')
base_w = Decimal('0.7')
mid_w = Decimal('0.4')
top_w = Decimal('0.25')
tip_y = Decimal('0.8')
tier_1_y = Decimal('0.5')
tier_2_y = Decimal('0.25')
base_y = Decimal('0.0')
trunk_bottom_y = -trunk_h

def create_tree_polygon(center_x, center_y, angle):
    initial_polygon = Polygon([
        (0.0, 0.8),
        (0.125, 0.5), (0.0625, 0.5),
        (0.2, 0.25), (0.1, 0.25),
        (0.35, 0.0),
        (0.075, 0.0), (0.075, -0.2),
        (-0.075, -0.2), (-0.075, 0.0),
        (-0.35, 0.0),
        (-0.1, 0.25), (-0.2, 0.25),
        (-0.0625, 0.5), (-0.125, 0.5),
    ])
    rotated = affinity.rotate(initial_polygon, float(angle), origin=(0, 0))
    return affinity.translate(rotated, xoff=float(center_x), yoff=float(center_y))

def get_side_length(trees):
    all_polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

# Parse data
df['x_val'] = df['x'].str.lstrip('s').astype(float)
df['y_val'] = df['y'].str.lstrip('s').astype(float)
df['deg_val'] = df['deg'].str.lstrip('s').astype(float)
df['n'] = df['id'].str.split('_').str[0].astype(int)

# Calculate score for each N
scores = []
for n in range(1, 201):
    group = df[df['n'] == n]
    trees = [{'x': row['x_val'], 'y': row['y_val'], 'deg': row['deg_val']} for _, row in group.iterrows()]
    side = get_side_length(trees)
    score = side**2 / n
    scores.append({'n': n, 'side': side, 'score': score})
    if n <= 10 or n % 20 == 0:
        print(f'N={n:3d}: side={side:.6f}, score={score:.6f}')

total_score = sum(s['score'] for s in scores)
print(f'\\nTotal Score: {total_score:.6f}')